In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
column_list = spark.sql(f"""DESCRIBE TABLE {catalog_name}.udp_wcm_bronze_sap_bw.sap_bw_wcm_pmat_plant""").collect()

dynamic_query_temp = f"""CREATE OR REPLACE TEMP VIEW  temp_sap_pmat_plant AS  SELECT """
dynamic_query_raw = f"""CREATE OR REPLACE TEMP VIEW  raw_sap_pmat_plant AS  SELECT """

for column in column_list:
    dynamic_query_temp = dynamic_query_temp + '`'+column[0]+'` AS '+ str.replace(column[0], '/','_') + ' ,'
dynamic_query_temp = dynamic_query_temp[:-1] + f""" FROM {catalog_name}.udp_wcm_bronze_sap_bw.sap_bw_wcm_pmat_plant
                WHERE proc_date =  CAST('{proc_date}' AS DATE)
"""


for column in column_list:
    dynamic_query_raw = dynamic_query_raw + '`'+column[0]+'` AS '+ str.replace(column[0], '/','_') + ' ,'
dynamic_query_raw = dynamic_query_raw[:-1] + f""" FROM {catalog_name}.udp_wcm_bronze_sap_bw.sap_bw_wcm_pmat_plant
                WHERE proc_date BETWEEN DATE_SUB(DATE('{proc_date}'),31) AND  DATE_SUB(DATE('{proc_date}'),1)"""

spark.sql(dynamic_query_temp)
spark.sql(dynamic_query_raw)

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_silver_sap_bw.sap_pmat_plant AS

WITH table_data AS (
 SELECT EXPLODE(
    SEQUENCE(
      TO_DATE('2021-05-06'),
      DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
    )
  ) AS date_gen
),

t1 AS (
    SELECT * EXCEPT(RN),
      LEAD(DATE(proc_date)) OVER(PARTITION BY plant, mat_plant ORDER BY proc_date ASC) AS date_to
    FROM (SELECT plant, LTRIM('0', mat_plant) AS mat_plant, proc_date, DATE(proc_date) AS calday,
    ROW_NUMBER() OVER(PARTITION BY plant, mat_plant, DATE(proc_date) ORDER BY proc_date DESC) AS rn
  FROM (
    SELECT * FROM raw_sap_pmat_plant
    UNION ALL
    SELECT * FROM temp_sap_pmat_plant) AS a ) AS a
    WHERE rn = 1
),

t11 AS (
    SELECT 
      date_gen,
      proc_date
    FROM table_data
    LEFT JOIN(
        SELECT calday, max(proc_date) as proc_date FROM t1 GROUP BY calday
    ) t11 ON date_gen = calday
),

t2 AS (
  SELECT 
    b.plant,
    b.mat_plant,
    a.proc_date,
    a.date_gen AS start_date,
    IF(date_gen = b.calday,'ACTIVE', 'DELETE') AS status
  FROM t11 a
    INNER JOIN t1 b 
        ON a.date_gen >= calday AND (a.date_gen < b.date_to OR b.date_to IS NULL)
),

t3 AS (
  SELECT 
    farm_fingerprint(CONCAT(
      IFNULL(CAST(plant AS STRING), ''),
      IFNULL(CAST(mat_plant AS STRING), ''),
      IFNULL(CAST(status AS STRING), '')
    )) AS hash_id,
    *
  FROM t2
),

t4 AS (
  SELECT *,
         LAG(hash_id,1) OVER(PARTITION BY plant, mat_plant ORDER BY start_date) AS hash_id_prev
  FROM t3
),

t5 AS (
  SELECT * EXCEPT(hash_id_prev)
  FROM t4
  WHERE hash_id != hash_id_prev OR hash_id_prev IS NULL
),

t6 AS (
  SELECT *,
         LEAD(start_date) OVER(PARTITION BY plant, mat_plant ORDER BY start_date ASC) AS end_date,
         MAX(start_date) OVER(PARTITION BY plant, mat_plant) AS max_last_update_date
  FROM t5
),

t7 AS (
  SELECT hash_id,
    plant,
    mat_plant,
    proc_date,
    status,
    start_date,
    IFNULL(DATE_SUB(end_date, 1), IF(max_last_update_date = start_date, '2400-01-01', start_date)) AS end_date,
    IF(max_last_update_date = start_date AND status = 'ACTIVE',0,1) is_delete  
  FROM t6
)

SELECT * FROM t7;
""")